## Imports

In [1]:
import sys
import os
import importlib
from pathlib import Path

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import shapiro, kstest
from scipy import stats  

from sklearn.linear_model import LogisticRegression, Ridge, Lasso, ElasticNet, RidgeCV, LassoCV,ElasticNetCV, LogisticRegressionCV
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import make_scorer, accuracy_score, classification_report, f1_score, matthews_corrcoef, mean_squared_error,r2_score, roc_auc_score, roc_curve, auc, confusion_matrix, log_loss
from sklearn.datasets import make_classification
from sklearn.preprocessing import MinMaxScaler
from statsmodels.stats.diagnostic import kstest_normal
from timeit import default_timer as timer
from tqdm import tqdm  
from typing import Optional
from joblib import Parallel, delayed
import pickle

module_path = str(Path("../src/data").resolve())
if module_path not in sys.path:
    sys.path.append(module_path)

import LogRegFxF as LR
import preprocessing as prep

project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))

In [2]:
sys.path

['/home/lestrada/miniconda3/envs/tumor_type_clasifier/lib/python313.zip',
 '/home/lestrada/miniconda3/envs/tumor_type_clasifier/lib/python3.13',
 '/home/lestrada/miniconda3/envs/tumor_type_clasifier/lib/python3.13/lib-dynload',
 '',
 '/home/lestrada/miniconda3/envs/tumor_type_clasifier/lib/python3.13/site-packages',
 '/home/lestrada/tumor_type_prediction/src/data']

In [8]:
import LogRegFxF as LR
import preprocessing as prep
import feature_selection as fs

In [39]:
importlib.reload(fs)

dir(fs)

['ElasticNet',
 'ElasticNetCV',
 'GridSearchCV',
 'Lasso',
 'LassoCV',
 'LogisticRegression',
 'LogisticRegressionCV',
 'MinMaxScaler',
 'Ridge',
 'RidgeCV',
 'StandardScaler',
 'StratifiedKFold',
 '__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__spec__',
 'accuracy_score',
 'auc',
 'binary_labeling',
 'chi2',
 'classification_report',
 'confusion_matrix',
 'cross_val_score',
 'f1_score',
 'fdrcorrection',
 'get_high_confidence_proteins',
 'hparameter_grid_search',
 'importlib',
 'kstest',
 'kstest_normal',
 'make_classification',
 'make_scorer',
 'matthews_corrcoef',
 'mean_squared_error',
 'np',
 'pd',
 'precision_recall_curve',
 'r2_score',
 'roc_auc_score',
 'roc_curve',
 'shapiro',
 'timer',
 'train_test_split']

## Data Import

In [5]:
#Proteins quantification intensities file
processed_data = '2024.10.23_CJ_pancancer_250/'
folder_path = '/media/kusterlab/internal_projects/active/TOPAS/WP31/Playground/Retrospective_study/'
PREPROCESSED_FP_INTENSITY = 'preprocessed_fp_with_ref.csv'
intensity_path_file = folder_path + processed_data + PREPROCESSED_FP_INTENSITY
input_quantifications = pd.read_csv(intensity_path_file)

#--------------------------------------------------------------------------------

#Samples metadata (oncotree classification) file.
metadata_path = '/media/kusterlab/internal_projects/active/TOPAS/WP31/Playground/Retrospective_MTBs_Evaluation/'
metadata_file = 'METADATA_PAN_CANCER_Batch300.xlsx'
the_metadata_file = metadata_path + metadata_file
input_metadata = pd.read_excel(the_metadata_file)

#--------------------------------------------------------------------------------

# # Proteins quantification z-scores file
# processed_data = '2024.10.23_CJ_pancancer_250/'
# folder_path = '/media/kusterlab/internal_projects/active/TOPAS/WP31/Playground/Retrospective_study/'
# PREPROCESSED_FP_INTENSITY = 'full_proteome_measures_z.tsv'
# intensity_path_file = folder_path + processed_data + PREPROCESSED_FP_INTENSITY
# df_Z_scores = pd.read_csv(intensity_path_file, sep='\t')

## Data Preprocessing

In [6]:
#Peptides quantification intensities post-processing

# Protein quantification intensities post-processing
input_quantifications = input_quantifications.set_index(input_quantifications.columns[0])
peptides_quant_info = prep.post_process_meta_intensities(input_quantifications.iloc[:,int(input_quantifications.shape[1]/2):].T ) #clean dataframe from regex characers
proteins_quant = input_quantifications.iloc[:,:int(input_quantifications.shape[1]/2)].T #subset protein measurements from dataset

#Imputation
prot_quant_imputed = prep.impute_normal_down_shift_distribution(proteins_quant) #Imputation of missing values in protein intensities using normal distribution down-shift method
na_columns = prot_quant_imputed.isna().any()
na_columns_true = na_columns[na_columns].index.tolist()
print("Proteins with  empty values:", na_columns_true)

#Cleaning sample names
prot_quant_imputed.reset_index(inplace=True)
prot_quant_imputed.rename(columns={'index': 'Sample name'}, inplace=True)
prot_quant_imputed['Sample name'] = prot_quant_imputed['Sample name'].str.replace('pat_', '')

#Dataset with protein intensities and metadata
samples_metadata = input_metadata[["Sample name", "code_oncotree",]] #sample metadata e.g. class, TCC, tissue of origin, etc.
initial_df = samples_metadata.merge(prot_quant_imputed, left_on='Sample name', right_on='Sample name')

#Peptides quantification to binary dataset
peptides_df_binary = pd.DataFrame(
    np.where(peptides_quant_info > 1, 1, 0), #if the # of peptides > 1, then turns to 1, otherwise 0. 
    index=peptides_quant_info.index,
    columns=peptides_quant_info.columns  
)
peptides_df_binary.reset_index(inplace=True) #Moves the index to a column. Allows to obtain patient id
peptides_df_binary.replace('Identification metadata ','',regex=True, inplace=True) #Removes text from id's
peptides_df_binary = samples_metadata.merge(peptides_df_binary, left_on='Sample name', right_on='index') #merging both data sets by Sample Name, ontaining a dataset with sample, classification and peptide binary count
peptides_df_binary.drop('index', axis=1, inplace=True)

peptides_df_binary

(2135, 13017)


/home/lestrada/miniconda3/envs/tumor_type_clasifier/lib/python3.13/site-packages/numpy/lib/_nanfunctions_impl.py:2015: RuntimeWarning: Degrees of freedom <= 0 for slice.
  var = nanvar(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
/home/lestrada/tumor_type_prediction/src/data/preprocessing.py:50: RuntimeWarning: Mean of empty slice
  temp_mean = np.nanmean(temp)


Proteins with  empty values: ['PTGER4', 'CD19', 'FOXO4', 'CRYGA', 'HNRNPCL3;HNRNPCL4', 'MYBPHL']


,Sample name,code_oncotree,MSH6,PCLAF,UTP18,SEC16A,IPO7,EIF3L,RPAP3,INTS3,...,ROPN1L,CARD10,ZNF804A,ZNF503,HHEX,STK40,FAM214A,WNT10B,VMO1,CCDC152
0,H021-3RLVZS-T1-Q1,AASTR,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
1,H021-VFM3B1-T1-Q1,AASTR,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
2,H021-3RLVZS-T1-Q1-R2,AASTR,1,0,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
3,H021-XBLS3R-M1-Q1,AASTR,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
4,H021-M2MSRE-M1-Q1,ACBC,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1655,H021-25HCGP-M2-Q1,VMM,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
1656,H021-VYS51F-M1-Q1,VSC,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
1657,H021-1B7R18-M1-Q1,VSC,1,0,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
1658,H021-FUFZFT-T1-Q1,VSC,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0


## Data Set Split

In [10]:
#Removing samples not part of the Oncotree classification
NOS_cases = ['CUPNOS', 'ADNOS', 'SARCNOS', 'SCCNOS', 'missing', 'SOLIDNOS', 'RCSNOS', 'GCTNOS']
ml_initial_df = prep.remove_class(initial_df, NOS_cases, 'code_oncotree')

# Splitting dataset into training and held-out sets1
training_df, held_out_df = prep.data_split(ml_initial_df, split_size=0.25, classified_by='code_oncotree', export=False)


Removed samples: 191
Remaining samples: 1469
Classes with only one sample: 70
Training set samples: 1119
Held-out set samples: 350


## Class Specific Worflow

In [12]:
ARMS_class = ['ARMS'] 
classified_by = 'code_oncotree'
samples_column = 'Sample name'

#Obtaining high confidence proteins by peptides
arms_proteins_by_peptides = fs.get_high_confidence_proteins(peptides_df_binary, ARMS_class, classified_by, threshold=0.7)

# Binary labeling for specific class classification - CREATE A FX or CLASS to do this alltogether with the following code
ARMS_training_df = fs.binary_labeling(training_df, classified_by=classified_by, true_class=ARMS_class)
ARMS_ho_df = fs.binary_labeling(held_out_df, classified_by=classified_by, true_class=ARMS_class)

# 1st Filter - Filtering ARMS training and held-out dataframes by proteins with peptides
ARMS_training_df = ARMS_training_df.filter(items=[samples_column, classified_by, 'Classifier'] + arms_proteins_by_peptides)
ARMS_ho_df = ARMS_ho_df.filter(items=[samples_column, classified_by, 'Classifier'] + arms_proteins_by_peptides)

 6336 proteins identified in 70.0% of ['ARMS'] samples

Number of samples per class:
Classifier
0    1076
1      43
Name: count, dtype: int64


Number of samples per class:
Classifier
0    336
1     14
Name: count, dtype: int64



## Feature Selection

In [40]:
ARMS_cv_results, ARMS_best_params, ARMS_best_score, ARMS_grid_search_obj = fs.hparameter_grid_search(ARMS_training_df.iloc[:, 0:20], 4, [0.3, 0.5, 0.7], [1, 10], classified_by='code_oncotree')

Grid search completed in 5.50 seconds
Best parameters: {'C': 10, 'l1_ratio': 0.3, 'max_iter': 10000, 'penalty': 'elasticnet', 'solver': 'saga'}
Best score: 0.6691314367649908


In [41]:
pd.DataFrame(ARMS_cv_results)

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_C,param_l1_ratio,param_max_iter,param_penalty,param_solver,params,...,split3_test_score,mean_test_score,std_test_score,rank_test_score,split0_train_score,split1_train_score,split2_train_score,split3_train_score,mean_train_score,std_train_score
0,0.428089,0.020077,0.004728,0.000135,1,0.3,10000,elasticnet,saga,"{'C': 1, 'l1_ratio': 0.3, 'max_iter': 10000, '...",...,0.463325,0.566526,0.100939,5,0.500104,0.630004,0.609204,0.599432,0.584686,0.050066
1,0.473568,0.021114,0.004486,0.000076,1,0.5,10000,elasticnet,saga,"{'C': 1, 'l1_ratio': 0.5, 'max_iter': 10000, '...",...,0.463325,0.566526,0.100939,5,0.500104,0.654182,0.589690,0.599432,0.585852,0.055274
2,0.561638,0.033125,0.005875,0.001903,1,0.7,10000,elasticnet,saga,"{'C': 1, 'l1_ratio': 0.7, 'max_iter': 10000, '...",...,0.463325,0.584284,0.103797,4,0.529237,0.633856,0.614734,0.623695,0.600380,0.041628
3,0.966783,0.117654,0.007096,0.001863,10,0.3,10000,elasticnet,saga,"{'C': 10, 'l1_ratio': 0.3, 'max_iter': 10000, ...",...,0.635987,0.669131,0.107330,1,0.621581,0.785123,0.770278,0.757923,0.733727,0.065459
4,1.051230,0.089570,0.008099,0.003444,10,0.5,10000,elasticnet,saga,"{'C': 10, 'l1_ratio': 0.5, 'max_iter': 10000, ...",...,0.635987,0.669131,0.107330,1,0.621581,0.785123,0.770278,0.757923,0.733727,0.065459
5,1.079541,0.156571,0.004615,0.001569,10,0.7,10000,elasticnet,saga,"{'C': 10, 'l1_ratio': 0.7, 'max_iter': 10000, ...",...,0.635987,0.669131,0.107330,1,0.668295,0.785123,0.790325,0.777654,0.755349,0.050462


In [ ]:
#cross validation for final model
def nested_cross_validation_logistic_regression(train_df:pd.DataFrame,classified_by:str, random_state=93):
    """
    classification_criteria: insert the 
    """
    y = train_df['Classifier']  # True values (dependent variable)
    X = train_df.drop(columns=['Sample name', classified_by, 'Classifier'], inplace=True)  # Independent variables (proteins)

    # Define the hyperparameter grid for Logistic Regression
    param_grid = {'C': [0.1, 1, 10]}

    # Set up the Logistic Regression model with L2 regularization (Ridge)
    logreg = LogisticRegression(penalty='elasticnet',
                                solver='saga',
                                l1_ratio=0,  # Ridge regularization
                                max_iter=10000,
                                class_weight='balanced',
                                warm_start=True)

    # Set up MCC scorer
    mcc_scorer = make_scorer(matthews_corrcoef)

    # Set up inner and outer cross-validation
    inner_cv = StratifiedKFold(n_splits=4, shuffle=True, random_state=random_state)
    outer_cv = StratifiedKFold(n_splits=4, shuffle=True, random_state=random_state)

    # Lists to store results
    outer_scores = []
    best_params = []
    inner_fold_cycle = 1
    # Perform nested cross-validation
    for train_idx, test_idx in outer_cv.split(X, y):
        X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
        y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]

        # Inner cross-validation with GridSearchCV
        grid_search = GridSearchCV(estimator=logreg, param_grid=param_grid, cv=inner_cv, scoring=mcc_scorer)
        grid_search.fit(X_train, y_train)

        # Get the best parameters and the score for the inner CV
        best_param = grid_search.best_params_
        best_score = grid_search.best_score_

        # Evaluate the best model on the outer test set
        best_model = grid_search.best_estimator_
        y_pred = best_model.predict(X_test)
        outer_score = matthews_corrcoef(y_test, y_pred)

        # Store the results
        outer_scores.append(outer_score)
        best_params.append(best_param)

        # Print the best parameters and the cross-validation score for each fold
        print(f"{inner_fold_cycle} Inner fold best parameter={best_param}, Score={best_score:.4f}, Outer MCC Score(held-out): {outer_score:.4f}")
        inner_fold_cycle += 1
 
    # Print overall mean MCC score
    print(f"Average MCC across all outer folds: {np.mean(outer_scores):.4f}")
    
    return outer_scores, best_params
